# Построение модели для предложения подходящего тарифа сотовой связи

### Описание проекта

**Цели и задачи:**

* Построить модель для задачи классификации, которая выберет подходящий для пользователей новый тариф сотовой связи.
* Оптимизировать модель по метрике accuracy (минимально допустимое значение 0.75).
* Проверить accuracy на тестовой выборке.

**Описание данных:**

Каждый объект в наборе данных users_behavior.csv — это информация о поведении одного пользователя за месяц.

* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

### Структура проекта

1. Изучение общей информации о данных
2. Построение моделей классификации
    * 2.1 Решающее дерево
    * 2.2 Случайный лес
    * 2.3 Логистическая регрессия
    * 2.4 Проверка модели на тестовой выборке
3. Проверка вменяемости модели
4. Общий вывод

## 1. Изучение общей информации о данных

In [1]:
# импортируем необходимые для работы библиотеки
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# открываем файл с данными
pth1 = 'C:\\Users\\Anna\\Desktop\\Python\\Yandex Praktikum\\Datasets\\project_6\\users_behavior.csv'

pth2 = '/datasets/users_behavior.csv'

if os.path.exists(pth1):
    users = pd.read_csv(pth1)
elif os.path.exists(pth2):
    users = pd.read_csv(pth2)
else:
    print('Something is wrong')

In [3]:
# выведем первые 10 строк таблицы с помощью метода head()
display(users.head(10))
# с помощью метода info() изучим структуры таблицы: типы данных, количество строк, столбцов, пропущенных данных.
users.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


* В таблице данных **3214** объектов. Пропусков нет.
* В столбцах `'calls'` и `'messages'` содержится целочисленная информация. Заменим тип данных в них на `int` для уменьшения занимаемого места.

In [4]:
users['calls'] = users['calls'].astype('int')
users['messages'] = users['messages'].astype('int')

## 2. Построение моделей классификации

Для начала выделим в исходных данных:
* признаки: `calls`, `minutes`, `messages`, `mb_used`
* целевой признак: `is_ultra`

In [5]:
features = users.drop('is_ultra', axis=1)
target = users['is_ultra']

Разделим данные на обучающую, валидационную и тестовую выборки в пропорции 3:1:1 соответственно (60% на обучающию и по 20% данных на валидационную и тестовую выборки).

In [6]:
# разделение проведём с помощью функции train_test_split библиотеки sklearn
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=1505)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=1505)

### 2.1 Решающее дерево

Инициируем модель решающего дерева `DecisionTreeClassifier`.

Выделим гиперпараметры для настройки модели:
* `max_depth` - максимальная глубина древа
* `min_samples_split` - минимальное количество объектов в узле
* `min_samples_leaf` - минимальное количество объектов в листе

Критерием проверки качества модели будет точность `accuracy`.

In [8]:
# в параметрах best_params и best_accuracy будем хранить наилучшие параметры для модели и наивысшую точность соответственно
best_params = []
best_accuracy = 0
# проходим в циклах по всем выбранным гиперпараметрам
for max_depth in range(2,10):
    for min_samples_split in range(2,6):
        for min_samples_leaf in range(1,10):
            # инициация модели с текущими гиперпараметрами
            model_dtc = DecisionTreeClassifier(random_state=1505,
                                               max_depth=max_depth,
                                               min_samples_split=min_samples_split,
                                               min_samples_leaf=min_samples_leaf)
            # обучение модели на тренировочной выборке
            model_dtc.fit(features_train, target_train)
            # поиск предсказаний модели на валидационной выбоке
            predictions = model_dtc.predict(features_valid)
            # вычисление точности модели методом accuracy_score
            accuracy = accuracy_score(target_valid, predictions)
            # если текущее значение точности выше предыдущего лучшего значения, 
            # сохраняем параметры модели и текущую точность
            if accuracy > best_accuracy:
                best_params = [max_depth, min_samples_split, min_samples_leaf]
                best_accuracy = accuracy

In [9]:
# вывод наилучшей точности и параметров модели.
print('''Наилучшая точность: {}
Парамтеры модели:
   max_depth = {}
   min_samples_split = {}
   min_samples_leaf = {}'''.format(round(best_accuracy, 2),
                             best_params[0],
                             best_params[1],
                             best_params[2],
                            ))

Наилучшая точность: 0.8
Парамтеры модели:
   max_depth = 9
   min_samples_split = 2
   min_samples_leaf = 3


Данная модель уже удовлетворяет условиям технического задания. Попробуем улучшить результат с другими моделями.

### 2.2 Случайный лес

Инициируем модель случайного леса `RandomForestClassifier`.

Выделим гиперпараметры для настройки модели:
* `n_estimators` - количество деревьев
* `max_depth` - максимальная глубина древа
* `min_samples_split` - минимальное количество объектов в узле
* `min_samples_leaf` - минимальное количество объектов в листе

Критерием проверки качества модели будет точность `accuracy`.

In [10]:
# в параметрах best_params и best_accuracy будем хранить наилучшие параметры для модели и наивысшую точность соответственно
best_params = []
best_accuracy = 0
# проходим в циклах по всем выбранным гиперпараметрам
for n_estimators in (1,10):
    for max_depth in range(2,10):
        for min_samples_split in range(2,10):
            for min_samples_leaf in range(1,8):
                # инициация модели с текущими гиперпараметрами
                model_rfc = RandomForestClassifier(random_state=1505,
                                                      n_estimators=n_estimators,
                                                      max_depth=max_depth,
                                                      min_samples_split=min_samples_split,
                                                      min_samples_leaf=min_samples_leaf)
                # обучение модели на тренировочной выборке
                model_rfc.fit(features_train, target_train)
                # поиск предсказаний модели на валидационной выбоке
                predictions = model_rfc.predict(features_valid)
                # вычисление точности модели методом accuracy_score
                accuracy = accuracy_score(target_valid, predictions)
                # если текущее значение точности выше предыдущего лучшего значения, 
                # сохраняем параметры модели и текущую точность
                if accuracy > best_accuracy:
                    best_params = [n_estimators, max_depth, min_samples_split, min_samples_leaf]
                    best_accuracy = accuracy

In [11]:
# вывод наилучшей точности и параметров модели.
print('''Наилучшая точность: {}
Парамтеры модели:
   n_estimators = {}
   max_depth = {}
   min_samples_split = {}
   min_samples_leaf = {}'''.format(round(best_accuracy, 2),
                             best_params[0],
                             best_params[1],
                             best_params[2],
                             best_params[3],
                            ))

Наилучшая точность: 0.82
Парамтеры модели:
   n_estimators = 10
   max_depth = 9
   min_samples_split = 7
   min_samples_leaf = 1


Точность в модели случайного леса удалось увеличить на 2% по сравнению с решающим деревом.

### 2.3 Логистическая регрессия

Инициируем модель логистической регрессии `LogisticRegression`.

Выделим гиперпараметры для настройки модели:
* `solver` - алгоритм построения модели

Критерием проверки качества модели будет точность `accuracy`.

In [12]:
model_lrc = LogisticRegression(random_state=1505, solver='lbfgs')
# обучение модели на тренировочной выборке
model_lrc.fit(features_train, target_train)
# поиск предсказаний модели на валидационной выбоке
predictions = model_lrc.predict(features_valid)
# вычисление точности модели методом accuracy_score
accuracy = accuracy_score(target_valid, predictions)

In [13]:
# вывод наилучшей точности и параметров модели.
print("""Наилучшая точность: {}
Парамтеры модели:
   solver = 'lbfgs'""".format(round(accuracy, 2)))

Наилучшая точность: 0.74
Парамтеры модели:
   solver = 'lbfgs'


Логистическая регрессия даёт более низкую точность, чем решающее дерево и случайный лес.

### 2.4 Проверка модели на тестовой выборке

По критерию `accuracy` наилучшей точности удалось достичь с моделью Случайного леса

Наилучшая точность: **0.82**

Парамтеры модели: 
* n_estimators = 10
* max_depth = 9
* min_samples_split = 7
* min_samples_leaf = 1

Проведём обучение данной модели на всех доступных данных (обучающие + валидационные) и вычислим точность на тестовой выборке.

In [14]:
features_merged = features_train.append(features_valid)
target_merged = target_train.append(target_valid)

In [15]:
model_rfc = RandomForestClassifier(random_state=1505,
                                   n_estimators=10,
                                   max_depth=9,
                                   min_samples_split=7,
                                   min_samples_leaf=1)
model_rfc.fit(features_merged, target_merged)
predictions = model_rfc.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
print('Точность модели на тестовой выборке: {}'.format(round(accuracy, 2)))

Точность модели на тестовой выборке: 0.8


**Выводы**

* Были проанализированы различные классификационные модели: Решающее дерево, Случайный лес, Логистическая регрессия.
* Каждая модель была обучена для различных сочетаний гиперпараметров, выведены наилучшие варианты для каждой модели.
* Наилучшая доля правильных ответов (accuracy) была достигнута для модели Случайного леса:
    * `Accuracy` на валидационной выборке: **0.82**
    * `Accuracy` на тестовой выборке: **0.80**
    * На тестовой выборке значение `Accuracy` упало на **0.02**

## 5. Проверка вменяемости модели

Проверим модель на адекватность двумя способами.

**1)** Для проверки адекватности модели сравним точность её предсказания с точностью константы:

In [16]:
const = (users['is_ultra'].value_counts() / users.shape[0]).loc[0]
if const < 0.79:
    print('Модель адекватна')
else:
    print('Модель неадекватна')

Модель адекватна


**2)** По другой версии, чтобы оценить адекватность модели в задачах классификации, нужно сравнить её со случайной.

У нас есть два значения целевого признака `is_ultra` - 1 и 0. Если бы модель предсказывала одно из двух чисел случайным образом, значение ее точности было бы **0.5**. С моделью `RandomForestClassifier` мы добились точности **0.8**, что гораздо выше.

Следовательно, модель прошла проверку на адекватность.

## 6. Общий вывод

**1)** Данные успешно загружены и проанализированы.
Пропусков не обнаружено, типы данных преобразованы в соответствии с хранимой информацией.

**2)** Были проанализированы различные классификационные модели: Решающее дерево, Случайный лес, Логистическая регрессия. Модели обучены на различных сочетаниях гиперпараметров.

* Наилучшая точность (accuracy) была достигнута для модели Случайного леса:
    * `Accuracy` на тестовой выборке: **0.8**

**3)** Точность Модели 'Случайный лес' была проанализирована двумя способами:
* сравнение с константой;
* сравнение со случайной моделью.

В обоих случаях модель прошла проверку на адекватность.